In [2]:
!pip install tensorflow 

import numpy as np
import os, json
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers, callbacks


  Using cached tensorboard-2.20.0-py3-none-any.whl.metadata (1.8 kB)
   ---------------------------------------- 0.0/332.0 MB ? eta -:--:--
   ---------------------------------------- 1.0/332.0 MB 8.9 MB/s eta 0:00:38
    --------------------------------------- 6.6/332.0 MB 20.9 MB/s eta 0:00:16
   - -------------------------------------- 13.4/332.0 MB 25.8 MB/s eta 0:00:13
   -- ------------------------------------- 20.7/332.0 MB 28.5 MB/s eta 0:00:11
   --- ------------------------------------ 27.8/332.0 MB 29.5 MB/s eta 0:00:11
   ---- ----------------------------------- 35.4/332.0 MB 31.0 MB/s eta 0:00:10
   ----- ---------------------------------- 41.9/332.0 MB 31.5 MB/s eta 0:00:10
   ------ --------------------------------- 49.8/332.0 MB 32.1 MB/s eta 0:00:09
   ------ --------------------------------- 57.4/332.0 MB 32.4 MB/s eta 0:00:09
   ------- -------------------------------- 63.2/332.0 MB 32.1 MB/s eta 0:00:09
   -------- ------------------------------- 67.9/332.0 MB 31.2 

In [3]:
BASE_PATH = 'images/images'  
TRAIN_DIR = os.path.join(BASE_PATH, 'train')
VAL_DIR   = os.path.join(BASE_PATH, 'validation')
BATCH_SIZE = 32
SEED = 42
CLASS_MODE = 'categorical'


In [4]:
# Basic generator (48x48)
IMG_SIZE_BASIC = (48, 48)
train_datagen_basic = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [5]:
val_datagen_basic = ImageDataGenerator(rescale=1./255)

train_gen_basic = train_datagen_basic.flow_from_directory(
    TRAIN_DIR, target_size=IMG_SIZE_BASIC, batch_size=BATCH_SIZE, class_mode=CLASS_MODE, seed=SEED)
val_gen_basic = val_datagen_basic.flow_from_directory(
    VAL_DIR, target_size=IMG_SIZE_BASIC, batch_size=BATCH_SIZE, class_mode=CLASS_MODE, shuffle=False)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


In [6]:
class_indices = train_gen_basic.class_indices
with open('class_indices.json', 'w') as f:
    json.dump(class_indices, f)

num_classes = train_gen_basic.num_classes

In [7]:
# -------- Basic CNN model ----------
def build_basic_cnn(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(64, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Conv2D(128, (3,3), activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2,2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

In [8]:
basic_cnn = build_basic_cnn(input_shape=IMG_SIZE_BASIC + (3,), num_classes=num_classes)
basic_cnn.compile(optimizer=optimizers.Adam(1e-3), loss='categorical_crossentropy', metrics=['accuracy'])
basic_cnn.summary()

C:\Users\USER\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 46, 46, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 46, 46, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 23, 23, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 21, 21, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 21, 21, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 10, 10, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 8, 8, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 7)              │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 357,831 (1.37 MB)

 Trainable params: 357,127 (1.36 MB)

 Non-trainable params: 704 (2.75 KB)

In [11]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# ✅ Create a folder to save your models
os.makedirs("models", exist_ok=True)

# ✅ Correct save path
checkpoint_path = "models/basic_cnn_best.keras"  # new keras format (recommended)

# ✅ Define Callbacks
callbacks_basic = [
    ModelCheckpoint(filepath=checkpoint_path, monitor='val_accuracy', save_best_only=True, mode='max'),
    EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
]



In [12]:
history_basic = basic_cnn.fit(train_gen_basic, epochs=20, validation_data=val_gen_basic, callbacks=callbacks_basic)

Epoch 1/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 40s 45ms/step - accuracy: 0.4047 - loss: 1.5386 - val_accuracy: 0.3651 - val_loss: 1.6392
Epoch 2/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 38s 42ms/step - accuracy: 0.4556 - loss: 1.4249 - val_accuracy: 0.4159 - val_loss: 1.5000
Epoch 3/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 43s 48ms/step - accuracy: 0.4757 - loss: 1.3738 - val_accuracy: 0.5309 - val_loss: 1.2437
Epoch 4/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 39s 43ms/step - accuracy: 0.4933 - loss: 1.3229 - val_accuracy: 0.4917 - val_loss: 1.3676
Epoch 5/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 40s 44ms/step - accuracy: 0.5095 - loss: 1.2897 - val_accuracy: 0.4577 - val_loss: 1.3865
Epoch 6/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 41s 45ms/step - accuracy: 0.5216 - loss: 1.2669 - val_accuracy: 0.5324 - val_loss: 1.2318
Epoch 7/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 40s 45ms/step - accuracy: 0.5269 - loss: 1.2468 - val_accuracy: 0.5331 - val_loss: 1.2370
Epoch 8/20
901/901 ━━━━━━━━━━━━━━━━━━━━ 42s 47ms/step - accuracy: 0.5361 - loss: 1.2248 - 